<a href="https://colab.research.google.com/github/juanFFlorezM/Extraccion_de_datos/blob/main/Analisis_Anova_PS_12_instructores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
# Paso 1: Cargar el archivo Excel
archivo_excel = 'Qt_3PS_excel.xlsx'  # Cambia esto por la ruta a tu archivo
datos = pd.read_excel(archivo_excel)

In [3]:
# Paso 2: Verifica la estructura de los datos
print("Datos cargados:")
print(datos.head())

Datos cargados:
  Instructor  S1  S2  S3
0         Q1   7   7   7
1         Q1   5   5   5
2         Q1   1   1   1
3         Q1   5   6   5
4         Q1   6   6   6


In [4]:
# Paso 3: Transformar los datos a formato largo
# Usamos pd.melt para transformar el DataFrame
datos_largos = pd.melt(datos, id_vars=['Instructor'], value_vars=['S1', 'S2', 'S3'],
                       var_name='Pregunta', value_name='Valor')

In [5]:
# Paso 4: Verifica la transformación
print("\nDatos transformados a formato largo:")
print(datos_largos.head())


Datos transformados a formato largo:
  Instructor Pregunta  Valor
0         Q1       S1      7
1         Q1       S1      5
2         Q1       S1      1
3         Q1       S1      5
4         Q1       S1      6


In [ ]:
# Paso 5: Realizar ANOVA
modelo = ols('Valor ~ C(Instructor)', data=datos_largos).fit()  # Ajusta el modelo OLS
anova_resultado = anova_lm(modelo, typ=2)                      # Realiza ANOVA
print("\nResultados de ANOVA:")
print(anova_resultado)


Resultados de ANOVA:
                   sum_sq     df         F    PR(>F)
C(Instructor)   62.864785   11.0  3.643621  0.000052
Residual       912.860805  582.0       NaN       NaN


In [ ]:
# Paso 6: Interpretar el resultado ANOVA
p_value_anova = anova_resultado['PR(>F)'][0]
if p_value_anova < 0.05:
    print("Hay diferencias significativas entre los instructores (p < 0.05).")

Hay diferencias significativas entre los instructores (p < 0.05).


<ipython-input-17-532034d692fc>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value_anova = anova_resultado['PR(>F)'][0]


In [ ]:
# Paso 7: Realizar la prueba post-hoc Tukey HSD si hay diferencias significativas
if p_value_anova < 0.05:
    tukey_resultado = pairwise_tukeyhsd(endog=datos_largos['Valor'], groups=datos_largos['Instructor'], alpha=0.05)
    print("\nResultados de Tukey HSD:")
    print(tukey_resultado)
else:
    print("No hay diferencias significativas entre los instructores (p >= 0.05).")


Resultados de Tukey HSD:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    Q1    Q10   0.1343    1.0 -0.6056  0.8741  False
    Q1    Q11  -0.4663  0.748 -1.2642  0.3317  False
    Q1    Q12   0.1912 0.9998 -0.6258  1.0083  False
    Q1     Q2   0.1038    1.0 -0.6484  0.8559  False
    Q1     Q3   0.5861 0.5842 -0.3069  1.4792  False
    Q1     Q4   0.4459  0.525 -0.2089  1.1006  False
    Q1     Q5   0.8323 0.0415  0.0152  1.6494   True
    Q1     Q6  -0.2866 0.9951 -1.1505  0.5773  False
    Q1     Q7  -0.1917 0.9997 -0.9726  0.5893  False
    Q1     Q8  -0.0652    1.0 -0.8823  0.7519  False
    Q1     Q9   0.2242 0.9968 -0.4848  0.9332  False
   Q10    Q11  -0.6005 0.4559  -1.446   0.245  False
   Q10    Q12    0.057    1.0 -0.8066  0.9206  False
   Q10     Q2  -0.0305    1.0  -0.833   0.772  False
   Q10     Q3   0.4519 0.9143  -0.484  1.3877  False
   Q10     Q4   0.31

In [6]:
# Este es un codigo que me permite realizar un diagrama de cajas y bigotes con la data en datos_largos
import plotly.express as px
fig = px.box(datos_largos, x='Instructor', y='Valor', title='Diagrama de Caja y Bigotes')
fig.show()